In [7]:
import numpy as np
import pandas as pd

credits= pd.read_csv('tmdb_5000_credits.csv')
movies= pd.read_csv('tmdb_5000_movies.csv')
movies.head()
credits.head()

movies = movies.merge(credits,on ='title')

movies = movies[['title','movie_id','keywords','overview','genres','crew','cast']]

movies.isnull().sum()

movies.dropna(inplace=True)


import ast
ast.literal_eval
def convert(obj):
    L =[]
    for i in ast.literal_eval(obj):
          L.append(i['name'])
    return L   
movies['genres']= movies['genres'].apply(convert)



movies['keywords']= movies['keywords'].apply(convert)



def convert(obj):
    L =[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter !=3:
            L.append(i['name'])
            counter +=3
        else:
            break
    return L  
movies['cast'] = movies['cast'].apply(convert)



def fetch_director(obj):
    L =[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
           L.append(i['name'])
           break
    return L   
movies['crew']= movies['crew'].apply(fetch_director)

movies['overview']= movies['overview'].apply(lambda x:x.split())
movies.head()

movies['genres']= movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movies['keywords']= movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast']= movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew']= movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])

movies.head()

# #merging all coloumn
movies['tags']=movies['overview'] + movies['keywords'] + movies['genres'] + movies['crew'] + movies['cast']
movies.head()


# #making new coloumn
new_df= movies[['movie_id','title','tags']]


# #converting list to string
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


#aftervectors
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem(text):
    y =[]

    for i in text.split():
       y.append(ps.stem(i))
    return " ".join(y)

new_df['tags']= new_df['tags'].apply(stem)

# #converting_all text to lower case
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

# Assuming new_df is your DataFrame and 'tags' is a column in
vectors =cv.fit_transform(new_df['tags']).toarray()



# Assuming cv is a CountVectorizer object
cv.get_feature_names_out()

from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)


sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(i)

import pickle

pickle.dump(new_df.to_dict(), open('movie_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

C:\Users\win10\AppData\Local\Temp\ipykernel_2256\4089494284.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
C:\Users\win10\AppData\Local\Temp\ipykernel_2256\4089494284.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']= new_df['tags'].apply(stem)
C:\Users\win10\AppData\Local\Temp\ipykernel_2256\4089494284.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index